In [1]:
from function import *

/home/wmbio/anaconda3/envs/deepdep_paper/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/wmbio/anaconda3/envs/deepdep_paper/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/wmbio/anaconda3/envs/deepdep_paper/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/wmbio/anaconda3/envs/d

In [2]:
print(os.getcwd())
os.chdir("/home/wmbio/WORK/gitworking/Dependency_prediction/")
print(os.getcwd())

/home/wmbio/WORK/gitworking/Dependency_prediction/prediction/src
/home/wmbio/WORK/gitworking/Dependency_prediction


In [3]:
with tf.device('/cpu:0'):
    model_name = "model_ccl501_exp_0801"
    # model_saved = tf.keras.models.load_model("prediction/custom_model/%s.h5" % model_name)
    model_saved = models.load_model("prediction/custom_model/%s.h5" % model_name)
    
    # model_name = "model_paper"
    # model_saved = models.load_model("prediction/model/%s.h5" % model_name)
# model_paper is the full 4-omics DeepDEP model used in the paper
# user can choose from single-omics, 2-omics, or full DeepDEP models from the
# /data/full_results_models_paper/models/ directory

* **Predict Set load**

In [4]:
PREDICT_PATH = "preprocessing/DATA/2022-07-29/"

In [5]:
# load TCGA genomics data and gene fingerprints
data_mut, data_labels_mut, sample_names_mut, gene_names_mut = load_data_prediction(PREDICT_PATH + "predict_mut_prediction.txt")
data_exp, data_labels_exp, sample_names_exp, gene_names_exp = load_data_prediction(PREDICT_PATH + "predict_exp_prediction.txt")
data_cna, data_labels_cna, sample_names_cna, gene_names_cna = load_data_prediction(PREDICT_PATH + "predict_cna_prediction.txt")
data_meth, data_labels_meth, sample_names_meth, gene_names_meth = load_data_prediction(PREDICT_PATH + "predict_meth_prediction.txt")
data_fprint_DepOIs, data_labels_fprint, gene_names_fprint, function_names_fprint = load_data_prediction(PREDICT_PATH + "predict_fingerprint_prediction.txt")
print("\n\nDatasets successfully loaded.\n\n")



Datasets successfully loaded.




In [7]:
pd.DataFrame(data_exp)

,0,1,2,3,4,5,6,7,8,9,...,6006,6007,6008,6009,6010,6011,6012,6013,6014,6015
0,5.5948,1.4542,0.0144,0.2750,3.5435,3.3420,0.7137,0.2388,0.6508,3.3674,...,0.0141,3.7371,0.1948,2.6169,0.5883,3.0723,1.4847,4.65,0.4107,0.4151
1,5.1914,0.1890,0.0000,3.3505,2.7485,3.2373,1.1309,3.6904,3.4944,5.4912,...,0.0141,3.7371,0.1948,2.6169,0.5883,3.0723,1.4847,4.65,0.4107,0.4151
2,5.3618,0.2265,0.0144,0.0286,0.0144,1.2869,0.0286,1.8520,1.2869,4.7522,...,0.0141,3.7371,0.1948,2.6169,0.5883,3.0723,1.4847,4.65,0.4107,0.4151


In [6]:
batch_size = 300
first_to_predict = 3
# predict the first 10 samples for DEMO ONLY, for all samples please substitute 10 by data_mut_tcga.shape[0]
# prediction results of all 8238 TCGA samples can be found in /data/full_results_models_paper/predictions/

In [8]:
with tf.device('/cpu:0'):
    t = time.time()
    data_pred = np.zeros((first_to_predict, data_fprint_DepOIs.shape[0]))
    for z in np.arange(0, first_to_predict):
        data_pred_tmp = model_saved.predict([
            # data_mut[np.repeat(z, data_fprint_DepOIs.shape[0])],
            data_exp[np.repeat(z, data_fprint_DepOIs.shape[0])],
            # data_cna[np.repeat(z, data_fprint_DepOIs.shape[0])],
            # data_meth[np.repeat(z, data_fprint_DepOIs.shape[0])],
            data_fprint_DepOIs
        ], 
            batch_size=batch_size, verbose=0)
        data_pred[z] = np.transpose(data_pred_tmp)
        print("Unscreended sample %d predicted..." % z)

Unscreended sample 0 predicted...
Unscreended sample 1 predicted...
Unscreended sample 2 predicted...


In [9]:
# write prediction results to txt
data_pred_df = pd.DataFrame(data=np.transpose(data_pred), index=gene_names_fprint, columns=sample_names_mut[0:first_to_predict])

In [10]:
data_pred_df

,IPC298,LOUNH91,697
AARS2,-0.598703,-0.348848,-0.682092
AASDHPPT,-0.372834,-0.344013,-0.347587
ABCE1,-1.729119,-1.821973,-1.717104
ABCG1,-0.164801,-0.065012,-0.177922
ABHD11,-0.297646,-0.254907,-0.361583
...,...,...,...
ZNF384,-0.042220,-0.009582,-0.055962
ZNF407,-0.628823,-0.644714,-0.677912
ZNF521,-0.018793,0.031740,-0.037457
ZNHIT2,-1.848377,-1.731368,-1.910353


In [ ]:
model_saved.predict([
            # data_mut[np.repeat(z, data_fprint_DepOIs.shape[0])],
            data_exp[np.repeat(z, data_fprint_DepOIs.shape[0])],
            # data_cna[np.repeat(z, data_fprint_DepOIs.shape[0])],
            # data_meth[np.repeat(z, data_fprint_DepOIs.shape[0])],
            data_fprint_DepOIs
        ], 
            batch_size=batch_size, verbose=0)

In [ ]:
model_saved.predict([
            # data_mut[np.repeat(z, data_fprint_DepOIs.shape[0])],
            data_exp[np.repeat(z, data_fprint_DepOIs.shape[0])],
            # data_cna[np.repeat(z, data_fprint_DepOIs.shape[0])],
            # data_meth[np.repeat(z, data_fprint_DepOIs.shape[0])],
            data_fprint_DepOIs
        ], 
            batch_size=batch_size, verbose=0)

In [ ]:
data_pred_df.to_csv('custom_mut_exp.csv')

In [ ]:
# pd.DataFrame.to_csv(data_pred_df, path_or_buf="/results/predictions/tcga_predicted_data_%s_demo.txt" % model_name, sep='\t', index_label='CRISPR_GENE', float_format='%.4f')
# print("\n\nPrediction completed in %.1f mins.\nResults saved in /results/predictions/tcga_predicted_data_%s_demo.txt\n\n" % (
#     (time.time()-t)/60, model_name))